In [2]:
from docplex.mp.model import Model
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import random

# Clase instancia

class Instancia:
    def __init__(self, instance = "instancias/n20w120.001.txt"):
        self.instance_name = instance.split("/")[1]
        print(self.instance_name)
        # Parámetros
        self.M = 10000
        self.cust_no = None
        self.coord_x = None
        self.coord_y = None
        self.demand = None
        self.ready_time = None
        self.due_date = None
        self.service_time = None
        # Leer archivo
        self.problem = self.leer_archivo(instance)

    
    def leer_archivo(self,instance):
        # Lectura de instancias
        file_head = []
        cols_head = []
        instance_data = []
        with open(instance, "r") as f:
            file = f.readlines()
            file_head = file[0].replace("\n","").split("\t")[0].split(" ")
            file_head = "{} {} {}".format(file_head[1],file_head[5],file_head[6])
            cols_head = file[3].replace("\n","").split("\t")
            for line in file[6:-1]:
                splited_line = line.replace("\n","").split("\t")
                instance_data.append(splited_line)

        # Convertir en array
        instance_data = np.array(instance_data).astype(int)
        self.cust_no = instance_data[:,0]
        self.coord_x = instance_data[:,1]
        self.coord_y = instance_data[:,2]
        self.demand = instance_data[:,3]
        self.ready_time = instance_data[:,4]
        self.due_date = instance_data[:,5]
        self.service_time = instance_data[:,6]
        self.file_head = file_head
        
        
        self.V = instance_data[:,0]

        self.n = len(self.V)
        
        self.arcos = [(i,j) for i in self.V for j in self.V if i!=j]
        self.a = {i: instance_data[:,4][i-1] for i in self.V}
        self.b = {i: instance_data[:,5][i-1] for i in self.V}
        self.c = {(i,j): int(np.hypot(self.coord_x[i-1] - self.coord_x[j-1] , self.coord_y[i-1] - self.coord_y[j-1])) for i,j in self.arcos}

        return self.n, self.ready_time, self.due_date, self.c
        
        

    def __repr__(self):
        return("Instancia: {} con {} clientes y header {}".format(self.instance_name,len(self.demand), self.file_head))

class Modelo():
    def __init__(self):
        self.name = "TSPTW"

    def build(self,ins):
        # Modelo
        self.mdl = Model('TSPTW')

        # Variables de decisión
        self.x = self.mdl.binary_var_dict(ins.arcos,name= 'x')
        self.u = self.mdl.integer_var_dict(ins.V, name= 'u', lb=0)

        # Restricción 1: Funcion Objetivo
        self.mdl.minimize(self.mdl.sum(self.x[(i,j)]*ins.c[(i,j)] for i,j in ins.arcos))

        # Restricción 2: Un arco saliente de cada vertice
        for i in ins.V:
            self.mdl.add_constraint(self.mdl.sum(self.x[(i,j)] for j in ins.V if i!=j) == 1, ctname = "restriccion2: x_%d" %(i))

        # Restricción 3: Un arco saliente de cada vertice
        for i in ins.V:
            self.mdl.add_constraint(self.mdl.sum(self.x[(j,i)] for j in ins.V if i!=j) == 1, ctname = "restriccion3: x_%d" %(i))

        # Restricción 4: Subrutas MTZ
        for i,j in ins.arcos:
            if i!=1:
                self.mdl.add_constraint(self.u[i]-self.u[j] + ins.M*self.x[(i,j)] <= ins.M - ins.c[(i,j)], ctname = "restriccion4: x_%d_%d" %(i,j))

        # Restricción 5: Ventanas de Tiempo
        for i in ins.V:
            self.mdl.add_constraints([ins.a[i] <= self.u[i] ,self.u[i] <= ins.b[i]])

        # print(self.mdl.pprint_as_string())

    def solve(self):
        self.solucion = self.mdl.solve(log_output= True)

        print(self.mdl.get_solve_status())
        print(self.solucion)
        self.arcos_solucion = [i for i in ins.arcos if self.x[i].solution_value>0.9]
        print('solucion',self.arcos_solucion)



    def plot(self,ins):
        fig, ax = plt.subplots(figsize=(12,7))
        for i in range(len(ins.V)):
            if i!=0:
                ax.scatter(ins.coord_x[i],ins.coord_y[i],300,color="black",marker = 's',zorder=2)
                ax.annotate(str(i+1),xy=(ins.coord_x[i],ins.coord_y[i]),xytext=(ins.coord_x[i]-0.5,ins.coord_y[i]-0.5), color="white")
            else:
                ax.scatter(ins.coord_x[i],ins.coord_y[i],300,color="red",marker = 's',label='Depósito',zorder=2)
                ax.annotate(str(i+1),xy=(ins.coord_x[i],ins.coord_y[i]),xytext=(ins.coord_x[i]-0.5,ins.coord_y[i]-0.5), color="white")

        for i,j in self.arcos_solucion:
            plt.plot([ins.coord_x[i-1],ins.coord_x[j-1]],[ins.coord_y[i-1],ins.coord_y[j-1]],color='black',zorder=1)

        DPatch = mpatches.Patch(color='red',label='Deposito')
        ISolPatch = mpatches.Patch(color='black',label='Cliente')
        plt.legend(handles=[DPatch, ISolPatch])

        plt.xlabel("Coordenada x")
        plt.ylabel("Coordenada y")
        plt.title("Solución instancia")
        
        # plt.savefig("fig.png")
        plt.show()


ins = Instancia()
# modelo = Modelo()
# modelo.build(ins)
# modelo.solve()
# modelo.plot(ins)

#print(ins.problem)

n = ins.problem[0]
a = ins.problem[1]
b = ins.problem[2]
c = ins.problem[3]

# print(n)
# print(a)
# print(b)
# print(c)



n20w120.001.txt


In [3]:
#Datos previos
ciudades = [10,21,3,14,5,20,7,8,19,21,11,2,13,4,15,6,17,18,9,16,1]
distancias = c.copy()
ventanas = {}
for i in range(len(ciudades)):
    ventanas[ciudades[i]] = a[i], b[i]



def Local1Shift(ciudades, ventanas, distancias): #Ciudades es el conjunto de ciudades en orden del camino y ventanas es un diccionario con ciudad :(a,b)
    def fueraDeVentana(ciudades, ventanas, distancias): #tiene que devolver los nodos y sus pos, en el caso de que no cumplan sus ventanas
        u = 0 # tiempo acumulado en la ciudad i
        flag = False
        infactibles = {}
        for i in range(len(ciudades)-1,-1,-1): 
            if i == 0:
                if u + distancias[(ciudades[len(ciudades)-1],ciudades[i])] < ventanas[ciudades[i]][0]:
                    u = ventanas[ciudades[i]][0]
                else: 
                    u += distancias[(ciudades[len(ciudades)-1],ciudades[i])]             
                    if u <= ventanas[ciudades[i]][1]:
                        infactibles[ciudades[i]] = [False, u]
                    else:
                        infactibles[ciudades[i]] = [True, u]
                
            else: #i != 0 and i != len(ciudades)
                if u + distancias[(ciudades[i-1],ciudades[i])] < ventanas[ciudades[i]][0]: 
                    u = ventanas[ciudades[i]][0]
                else:
                    u += distancias[(ciudades[i-1],ciudades[i])]             
                    if u <= ventanas[ciudades[i]][1]:
                        infactibles[ciudades[i]] = [False, u]
                    else:
                        infactibles[ciudades[i]] = [True, u]

        return infactibles #False == factible, True == infactible

    def verificar(ciudades, ventanas, distancias,p): #p=1 si se queire obtener la pos del primer infactible #p=0 si se queire obtener la pos del primer factible 
        infactibles = fueraDeVentana(ciudades, ventanas, distancias)

        if p == 1:
            for i in infactibles:
                if infactibles[i][0] == True:
                    pos = ciudades.index(i)
                    flag = True #Con que solo una ciudad no cumpla, el camino es infactible
                    break #Se hace solo para el primer infactible
                else: #No hay infactibles
                    pos = None
                    flag = False
        else: #p==0
            for i in infactibles:
                if infactibles[i][0] == False:
                    pos = ciudades.index(i)
                    flag = True #Con que solo una ciudad no cumpla, el camino es infactible
                    break
                else:
                    pos = None
                    flag = True    
        return infactibles, pos, flag


    def backwardViolated(ciudades, ventanas, distancias):
        infactibles, pos, flag = verificar(ciudades, ventanas, distancias,1)
        if flag == True:
            if pos == 0:
                aux = ciudades[pos]
                ciudades[pos] = ciudades[len(ciudades)-1]
                ciudades[len(ciudades)-1] = aux
            
            if pos == None:
                pass

            else:
                pos2 = None
                for i in range(pos,-1,-1):
                    if i != pos:
                        if infactibles[ciudades[pos]][1] + distancias[(ciudades[i],ciudades[pos])] <= ventanas[ciudades[pos]][1]:
                            pos2 = i
                            break #El mas cercano al infactible que cumpla con el criterio de ventana
                        else:
                            pos2 = None
                if pos2 != None:
                    aux = ciudades[pos]
                    for i in range(pos,pos2,-1): #Se corren +1 las posiciones de las ciudades, queda la ciudad en la pos2 repetida
                        ciudades[i] = ciudades[i-1]
                    ciudades[pos2] = aux
        else:
            pass

        return ciudades

    def forwardNonViolated(ciudades, ventanas, distancias):
        infactibles, pos, flag = verificar(ciudades, ventanas, distancias,0)
        if flag == True:
            if pos == len(ciudades)-1:
                aux = ciudades[pos]
                ciudades[pos] = ciudades[0]
                ciudades[0] = aux

            if pos == None:
                pass

            else:
                pos2 = None
                for i in range(pos+1,len(ciudades)):
                    if i != pos:
                        if infactibles[ciudades[pos]][1] + distancias[(ciudades[i],ciudades[pos])] >= ventanas[ciudades[pos]][1]: #Queda fuera de la ventana
                            pos2 = i
                            # print('i',i,'pos', pos)
                            # print('pos2_1',pos2)
                            break #El mas cercano al infactible que cumpla con el criterio de ventana
                        else:
                            pos2 = None
                            # print('pos2_2',pos2)
                # print('pos2_3',pos2)       
                if pos2 != None:
                    aux = ciudades[pos]
                    #for i in range(pos,pos2,-1): #Se corren -1 las posiciones de las ciudades, queda la ciudad en la pos2 repetida
                    for j in range(pos,pos2):
                            ciudades[j] = ciudades[j+1]
                    ciudades[pos2] = aux
        else: 
            pass

        return ciudades
    
    def forwardViolated(ciudades, ventanas, distancias):
        infactibles, pos, flag = verificar(ciudades, ventanas, distancias,1)
        #print('flag5',flag)
        if flag == True:
            if pos == len(ciudades)-1:
                aux = ciudades[pos]
                ciudades[pos] = ciudades[0]
                ciudades[0] = aux
            if pos == None:
                pass
            else:
                pos2 = None    
                for i in range(pos+1,len(ciudades)):
                    if i != pos:
                        if infactibles[ciudades[pos]][1] + distancias[(ciudades[i],ciudades[pos])] >= ventanas[ciudades[pos]][1]: #Queda fuera de la ventana
                            pos2 = i
                            # print('pos2_1',pos2)
                            break #El mas cercano al infactible que cumpla con el criterio de ventana
                        else:
                            pos2 = None
                            # print('pos2_2',pos2)
                if pos2 != None:
                    aux = ciudades[pos]
                    #for i in range(pos,pos2,-1): #Se corren -1 las posiciones de las ciudades, queda la ciudad en la pos2 repetida
                    for j in range(pos,pos2):
                            ciudades[j] = ciudades[j+1]
                    ciudades[pos2] = aux
        else:
            pass

        return ciudades
    
    def backwardNonViolated(ciudades, ventanas, distancias):
        infactibles, pos, flag = verificar(ciudades, ventanas, distancias,0)
        if flag == True:
            if pos == 0:
                aux = ciudades[pos]
                ciudades[pos] = ciudades[len(ciudades)-1]
                ciudades[len(ciudades)-1] = aux
            if pos == None:
                pass
            else:
                pos2 = None
                for i in range(pos,-1,-1):
                    if i != pos:
                        if infactibles[ciudades[pos]][1] + distancias[(ciudades[i],ciudades[pos])] <= ventanas[ciudades[pos]][1]:
                            pos2 = i
                            # print('i',i,'pos', pos)
                            # print('pos2_1',pos2)
                            break #El mas cercano al infactible que cumpla con el criterio de ventana
                        else:
                            pos2 = None
                            # print('pos2_2',pos2)
                if pos2 != None:
                    aux = ciudades[pos]
                    for j in range(pos,pos2,-1): #Se corren +1 las posiciones de las ciudades, queda la ciudad en la pos2 repetida
                        ciudades[j] = ciudades[j-1]
                    ciudades[pos2] = aux
        else:
            pass 

        return ciudades
            
    #Main           
    it = 0 #nr de iteraciones max
    #it_max = len(ciudades)*len(ciudades)
    it_max = 1000

    infactibles, pos, flag = verificar(ciudades, ventanas, distancias,1)

    print('ciudades ini', ciudades)

    while flag == True :
        ciudades = backwardViolated(ciudades, ventanas, distancias)
        infactibles, pos, flag = verificar(ciudades, ventanas, distancias,1)
        #print('1',ciudades)
        ciudades = forwardNonViolated(ciudades, ventanas, distancias)
        infactibles, pos, flag = verificar(ciudades, ventanas, distancias,1)
        #print('2',ciudades)
        ciudades = backwardNonViolated(ciudades, ventanas, distancias)
        infactibles, pos, flag = verificar(ciudades, ventanas, distancias,1)
        #print('3',ciudades)
        ciudades = forwardViolated(ciudades, ventanas, distancias)
        infactibles, pos, flag = verificar(ciudades, ventanas, distancias,1)
        #print('4',ciudades)
        it += 1
        if flag == False:
            print('FACTIBLE')
        if it == it_max:
            break
        #print('it',it)
    print(ciudades)


ciudades = Local1Shift(ciudades, ventanas, distancias)


ciudades ini [10, 21, 3, 14, 5, 20, 7, 8, 19, 21, 11, 2, 13, 4, 15, 6, 17, 18, 9, 16, 1]
[9, 21, 3, 14, 5, 20, 7, 8, 19, 21, 11, 2, 13, 15, 4, 6, 17, 10, 18, 1, 16]
